# SDP Schedule Simulation

In [ ]:
import sys
sys.path.insert(0, "..")

from sdp_par_model.scheduling.simulation import ScheduleSimulation

In [ ]:
app = ScheduleSimulation()

## Overall observatory selection & capacities

In [ ]:
scenario = "low-adjusted"

app.observatory_sizes_and_rates(scenario)

## Read HPSO performance characteristics

Loads high performance science objective characteristics generated by the export notebook. This picks up the latest file checked into Git by default, but a csv file path can be specified.

In [ ]:
app.read_hpso_csv(csv_file=None)

## Determine computational capacity required for realtime processing

As SKA SDP needs to be able to change observation at arbitrary times, we need to always reserve enough computational resources to deal with the most expensive case. Here we figure this out automatically based on the calculated parameters.

In [ ]:
app.computational_capacity()

## Generate graph

Generate a sequence with all HPSOs appearing roughly as often as we expect them in a real-life schedule. We then shuffle this list and generate a (multi-)graph of tasks from it.

Note that in contrast to Francois' scheduler, the resource usage of every task is fixed up-front, therefore we need to declare certain key sizes here. Adjust as necessary in relation to the capacities (see below) to get the desired amount of parallelism between tasks.

In [ ]:
Tsequence = 20 * 24 * 3600
Tobs_min = 10 * 60
batch_parallelism = 2
display_node_info = False

app.generate_graph(Tsequence, Tobs_min, batch_parallelism, display_node_info)

## Sanity-check

We can do a number of consistency checks at this point: Clearly we should have enough capacity to run every task in isolation.

Furthermore, in order to keep up with observations we need to make sure that we are not over-using any resource on average. This is a pretty rough estimate of safety that especially under-estimates the cost of edges in high-pressure scenarios. For example, if somethings needs to be kept in the buffer for longer, it has a higher footprint than estimated here. Therefore especially the size of `input-buffer` and `output-buffer` should be quite generous here.

In [ ]:
app.sanity_check()

## Schedule tasks

Assign a task time to every node, and figure out resource usages and edge lengths along the way.

In [ ]:
app.schedule_tasks()

## Efficiency calculations

We can play around with capacities and see how it affects overall efficiency. This takes quite a bit, so let's set up some multiprocessing infrastructure to take advantage of parallelism:

In [ ]:
app.efficiency_calculations()

## Dealing with failures

It is also possible to simulate failures. The idea is that we model this as a temporary capacity change, which requires us to re-schedule twice (once the capacity reduces, and once it's restored)

In [ ]:
app.failures()